<a href="https://colab.research.google.com/github/nicikess/hsg-ml-course/blob/main/notebooks/ml_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image

!pip install rasterio
import rasterio
from rasterio.plot import reshape_as_image

from datetime import datetime

from tifffile import imread

from sklearn.metrics import confusion_matrix
from sklearn import model_selection

import torch, torchvision
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import transforms
from torchvision.transforms import ToTensor

     |████████████████████████████████| 19.3 MB 345 kB/s 


In [5]:
#Loads tiff folders

import os
folder = '/content/drive/MyDrive/ML/data/dataset/trainset/'
subfolders = []
for dirs in os.walk(folder):
  subfolders.append(dirs[0])

#remove directory
subfolders.pop(0)
for i in range(0, len(subfolders)):
     print(subfolders[i])

/content/drive/MyDrive/ML/data/dataset/trainset/AnnualCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Forest
/content/drive/MyDrive/ML/data/dataset/trainset/HerbaceousVegetation
/content/drive/MyDrive/ML/data/dataset/trainset/Highway
/content/drive/MyDrive/ML/data/dataset/trainset/Industrial
/content/drive/MyDrive/ML/data/dataset/trainset/Pasture
/content/drive/MyDrive/ML/data/dataset/trainset/PermanentCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Residential
/content/drive/MyDrive/ML/data/dataset/trainset/River
/content/drive/MyDrive/ML/data/dataset/trainset/SeaLake


In [6]:
#Define paths

ROOT_PATH = '/content/drive/MyDrive/ML/'
BASE_PATH = os.path.join(ROOT_PATH, 'data')
MODEL_PATH = os.path.join(ROOT_PATH, 'model')

file_path_train = '/content/drive/MyDrive/ML/data/dataset/train.csv'
file_path_test = '/content/drive/MyDrive/ML/data/dataset/test.csv'
data_df_train = pd.read_csv(file_path_train)
data_df_test = pd.read_csv(file_path_test)

data_df_train.columns = data_df_train.columns.str.replace('path', 'image_id')
data_df_train.columns = data_df_train.columns.str.replace('Label', 'label')

In [7]:
#Init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

In [8]:
#Classes and labels

IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
}

CLASSES = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']
CLASS_IDX_LABELS = dict()
for key, val in IDX_CLASS_LABELS.items():
  CLASS_IDX_LABELS[val] = key

NUM_CLASSES = len(IDX_CLASS_LABELS.items())
torch.manual_seed(10)
VALID_SIZE = 0.1

In [9]:
#Encoder and decoder

## Give idx of each class name
def encode_label(label):
    idx = CLASS_IDX_LABELS[label] 
    return idx

## Take in idx and return the class name
def decode_target(target, text_labels=True):
    result = []
    if text_labels:
        return IDX_CLASS_LABELS[target]
    else:
        return target

In [10]:
#Open tiff files with rasterio and safe to dictionary

'''import os

data = {}

for i in range(len(subfolders)):

  txtfiles = []

  # traverse whole directory
  for root, dirs, files in os.walk(subfolders[i]):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            txtfiles.append(os.path.join(root, file))

  code = subfolders[i].split('/')[8]
  index = encode_label(code)
  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")

  print(code)
  print(index)

  i = 0
  for pic in txtfiles:
    with rasterio.open(pic, "r") as img:
      arr[i] = np.moveaxis(img.read(), 0, 2)
      i += 1
      
  data[index] = arr'''

'import os\n\ndata = {}\n\nfor i in range(len(subfolders)):\n\n  txtfiles = []\n\n  # traverse whole directory\n  for root, dirs, files in os.walk(subfolders[i]):\n    # select file name\n    for file in files:\n        # check the extension of files\n        if file.endswith(\'.tif\'):\n            txtfiles.append(os.path.join(root, file))\n\n  code = subfolders[i].split(\'/\')[8]\n  index = encode_label(code)\n  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")\n\n  print(code)\n  print(index)\n\n  i = 0\n  for pic in txtfiles:\n    with rasterio.open(pic, "r") as img:\n      arr[i] = np.moveaxis(img.read(), 0, 2)\n      i += 1\n      \n  data[index] = arr'

In [11]:
#Save the files as .npy

'''for i in range(len(data)):
  keys_list = list(data)
  key = keys_list[i]
  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)'''

"for i in range(len(data)):\n  keys_list = list(data)\n  key = keys_list[i]\n  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)"

In [12]:
#Reload the .npy files

import os
path = '/content/drive/MyDrive/ML/data/dataset/trainset/'
npyfiles = []
data = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in range(len(npyfiles)):
  path = npyfiles[i]
  arr = np.load(path)
  index = path.split('/')[8].split('_')[1][5]
  index = int(index)
  data[index] = arr

In [147]:
from pathlib import Path
class EuroSAT(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        row = self.train_df.loc[idx]
        img_id, label = row['image_id'], row['label']
        img_id = int(img_id.split('.')[0].split('_')[1])
        label_encode = int(encode_label(label))
        id = img_id - 1
        img = data[label_encode][id]
        img_rgb = img[:, :, [12,11,10,9,8,7,6,5,4,3,2,1]]
        if self.transform:
            img_rgb = self.transform(img_rgb)
        return img_rgb, encode_label(label)

In [148]:
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),transforms.ConvertImageDtype(torch.float)])

train_ds = EuroSAT(data_df_train, BASE_PATH, transf)
print(len(train_ds))

27000


In [149]:
np.shape(train_ds[0][0])
print(train_ds[0][0])

tensor([[[3432., 3432., 3630.,  ..., 2997., 3011., 3007.],
         [3432., 3432., 3630.,  ..., 2997., 3011., 3007.],
         [3401., 3401., 3585.,  ..., 3035., 3059., 3079.],
         ...,
         [3693., 3693., 3395.,  ..., 3381., 3329., 3323.],
         [3662., 3662., 3455.,  ..., 3405., 3344., 3330.],
         [3614., 3614., 3502.,  ..., 3429., 3358., 3335.]],

        [[ 654.,  654.,  627.,  ..., 2206., 2164., 2066.],
         [ 654.,  654.,  627.,  ..., 2206., 2164., 2066.],
         [ 659.,  659.,  633.,  ..., 2037., 1975., 1869.],
         ...,
         [1088., 1088., 1330.,  ..., 1381., 1798., 2156.],
         [ 977.,  977., 1128.,  ..., 1301., 1705., 2078.],
         [ 867.,  867.,  940.,  ..., 1230., 1614., 1998.]],

        [[1501., 1501., 1492.,  ..., 3544., 3469., 3292.],
         [1501., 1501., 1492.,  ..., 3544., 3469., 3292.],
         [1498., 1498., 1490.,  ..., 3299., 3208., 3041.],
         ...,
         [2039., 2039., 2241.,  ..., 2502., 3053., 3555.],
         [

Model

In [151]:
pretrainedBool = True
model = models.resnet50(pretrained=pretrainedBool)
if pretrainedBool:
    for param in model.parameters():
        param.requires_grad = False
model.fc = nn.Linear(model.fc.in_features, 10)
model.conv1 = nn.Conv2d(12, 64, kernel_size=7, stride=2, padding=3,bias=False)

In [152]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

[LOG] notebook with cuda computation enabled


In [153]:
model = model.to(device)

In [154]:
# define the optimization criterion / loss function
ce_loss = nn.CrossEntropyLoss()

In [155]:
ce_loss = ce_loss.to(device)

In [156]:
# define learning rate and optimization strategy
learning_rate = 0.001
params = model.fc.parameters() if pretrainedBool else model.parameters()
optimizer = torch.optim.Adam(params, lr=learning_rate, weight_decay=0)

In [160]:
# specify the training parameters
num_epochs = 20 # number of training epochs
mini_batch_size = 256 # size of the mini-batches

In [161]:
train_dl = DataLoader(train_ds, batch_size=mini_batch_size, shuffle=True)

In [177]:
import torch
torch.cuda.empty_cache()

In [163]:
# init collection of training epoch losses
train_epoch_losses = []

# set the model in training mode
model.train()

# train the CIFAR10 model
for epoch in range(num_epochs):
    
    # init collection of mini-batch losses
    train_mini_batch_losses = []

    # iterate over all-mini batches
    for i, (images, labels) in enumerate(train_dl):

        #print(images)

        # push mini-batch data to computation device
        images = images.to(device)
        labels = labels.to(device)

        # run forward pass through the network
        output = model(images)
        
        # reset graph gradients
        model.zero_grad()
        
        # determine classification loss
        loss = ce_loss(output, labels)
        
        # run backward pass
        loss.backward()
        
        # update network paramaters
        optimizer.step()
        
        # collect mini-batch reconstruction loss
        train_mini_batch_losses.append(loss.data.item())

    # determine mean min-batch loss of epoch
    train_epoch_loss = np.mean(train_mini_batch_losses)
    
    # print epoch loss
    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
    
    # set filename of actual model
    model_name = 'challenge_model_epoch_{}.pth'.format(str(epoch))

    # save current model to GDrive models directory
    torch.save(model.state_dict(), os.path.join(MODEL_PATH, model_name))
    
    # determine mean min-batch loss of epoch
    train_epoch_losses.append(train_epoch_loss)

[LOG 20220501-19:45:29] epoch: 0 train-loss: 0.9811363439514952
[LOG 20220501-19:45:52] epoch: 1 train-loss: 0.7141922076918045
[LOG 20220501-19:46:16] epoch: 2 train-loss: 0.6510201613858061
[LOG 20220501-19:46:39] epoch: 3 train-loss: 0.6177061593757486
[LOG 20220501-19:47:02] epoch: 4 train-loss: 0.5919699851634368
[LOG 20220501-19:47:25] epoch: 5 train-loss: 0.5785083557074925
[LOG 20220501-19:47:49] epoch: 6 train-loss: 0.5514891217902022
[LOG 20220501-19:48:12] epoch: 7 train-loss: 0.536738488471733
[LOG 20220501-19:48:35] epoch: 8 train-loss: 0.5307503584420906
[LOG 20220501-19:48:58] epoch: 9 train-loss: 0.5144846158207588
[LOG 20220501-19:49:22] epoch: 10 train-loss: 0.5080168019488173
[LOG 20220501-19:49:45] epoch: 11 train-loss: 0.5053917228613259
[LOG 20220501-19:50:08] epoch: 12 train-loss: 0.49444218148600383
[LOG 20220501-19:50:31] epoch: 13 train-loss: 0.4824814034520455
[LOG 20220501-19:50:55] epoch: 14 train-loss: 0.4839829271132091
[LOG 20220501-19:51:18] epoch: 15 t

In [164]:
import os
path = '/content/drive/MyDrive/ML/data/dataset/testset/'
npyfiles = []
datatest = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in range(len(npyfiles)):
  path = npyfiles[i]
  arr = np.load(path)
  index = int(path.split('_')[1].split('.')[0])
  datatest[index] = arr

In [165]:
from pathlib import Path
class EuroSATTest(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        img = datatest[idx]
        img = img[:, :, ]
        img = np.float32(img)
        if self.transform:
            img = self.transform(img)
        return img

In [166]:
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),transforms.ConvertImageDtype(torch.float)])
test_ds = EuroSATTest(data_df_test, BASE_PATH, transf)
print(len(test_ds))

4232


In [167]:
test_ds[1]

tensor([[[ 597.,  581.,  564.,  ...,  610.,  585.,  560.],
         [ 600.,  583.,  566.,  ...,  616.,  592.,  568.],
         [ 603.,  585.,  567.,  ...,  623.,  599.,  575.],
         ...,
         [ 371.,  373.,  375.,  ...,  413.,  423.,  434.],
         [ 383.,  386.,  390.,  ...,  406.,  416.,  426.],
         [ 395.,  400.,  404.,  ...,  398.,  409.,  419.]],

        [[ 449.,  616.,  518.,  ...,  840.,  554.,  574.],
         [ 416.,  515.,  541.,  ...,  752.,  947.,  546.],
         [ 486.,  582.,  505.,  ...,  496.,  786.,  411.],
         ...,
         [ 328.,  315.,  386.,  ...,  439.,  410.,  398.],
         [ 328.,  349.,  383.,  ...,  405.,  449.,  416.],
         [ 387.,  598.,  512.,  ...,  414.,  475.,  470.]],

        [[ 731.,  909.,  808.,  ..., 1156.,  929.,  818.],
         [ 636.,  745.,  782.,  ..., 1115., 1115.,  794.],
         [ 754.,  769.,  756.,  ...,  763., 1034.,  818.],
         ...,
         [ 635.,  561.,  743.,  ...,  843.,  756.,  651.],
         [

In [168]:
np.shape(test_ds[0])

torch.Size([12, 64, 64])

In [169]:
cifar10_eval_dataloader = DataLoader(test_ds, batch_size=10_000, num_workers=2, pin_memory=True)

In [171]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_19.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

<All keys matched successfully>

In [ ]:
# set model in evaluation mode
best_model.eval()

In [179]:
predictions = torch.argmax(best_model(iter(cifar10_eval_dataloader).next().to(device)), dim=1)

RuntimeError: ignored

In [63]:
list_tens = predictions.tolist()

for i in range(len(list_tens)):
  list_tens[i] = decode_target(list_tens[i])

In [64]:
print(list_tens)

['Forest', 'HerbaceousVegetation', 'Forest', 'SeaLake', 'HerbaceousVegetation', 'SeaLake', 'SeaLake', 'SeaLake', 'Forest', 'SeaLake', 'SeaLake', 'SeaLake', 'Forest', 'PermanentCrop', 'SeaLake', 'Forest', 'SeaLake', 'River', 'Forest', 'SeaLake', 'SeaLake', 'HerbaceousVegetation', 'SeaLake', 'SeaLake', 'SeaLake', 'Industrial', 'Forest', 'Forest', 'SeaLake', 'Highway', 'SeaLake', 'Forest', 'Forest', 'SeaLake', 'Forest', 'SeaLake', 'SeaLake', 'SeaLake', 'SeaLake', 'SeaLake', 'Forest', 'Forest', 'Highway', 'SeaLake', 'Forest', 'HerbaceousVegetation', 'SeaLake', 'SeaLake', 'Forest', 'Forest', 'HerbaceousVegetation', 'SeaLake', 'SeaLake', 'Forest', 'Industrial', 'Forest', 'HerbaceousVegetation', 'Forest', 'SeaLake', 'Forest', 'Forest', 'Forest', 'Forest', 'HerbaceousVegetation', 'SeaLake', 'SeaLake', 'SeaLake', 'SeaLake', 'AnnualCrop', 'SeaLake', 'SeaLake', 'Forest', 'SeaLake', 'SeaLake', 'Forest', 'Forest', 'SeaLake', 'SeaLake', 'SeaLake', 'SeaLake', 'Forest', 'Forest', 'Forest', 'Forest', '

In [65]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

dict_keys(['Forest', 'HerbaceousVegetation', 'SeaLake', 'PermanentCrop', 'River', 'Industrial', 'Highway', 'AnnualCrop', 'Residential', 'Pasture'])
dict_values([1737, 432, 1497, 88, 28, 225, 75, 88, 50, 12])


In [67]:
test_indices_path = '/content/drive/MyDrive/ML/data/dataset/test.csv'

final_submission_file = pd.read_csv(test_indices_path)
prediction_list = []

# turning numeric predictions to strings (as in kaggle example submission)
for p in predictions:
  prediction_list.append(decode_target(p.item()))

final_submission_file["label"] = prediction_list
### source with alternatives methods for appending to csv should this fail: https://stackoverflow.com/questions/26666919/add-column-in-dataframe-from-list
final_submission_file.to_csv('/content/drive/MyDrive/ML/data/dataset/final_submission.csv', index=None)